# Dataset de Tweets para Análisis de Sentimientos

## Capturamos un train y un test de tweets previamente seleccionados por una competencia

Los detalles de la competencia **gratuita** pueden encontrarse en la siguiente web: [https://datahack.analyticsvidhya.com/contest/practice-problem-twitter-sentiment-analysis/](https://)

In [0]:
import pandas as pd
train=pd.read_csv('https://raw.githubusercontent.com/javalpe/datasets/master/train_twitter_analysis.csv')
test=pd.read_csv('https://raw.githubusercontent.com/javalpe/datasets/master/test_twitter_analysis.csv')

## Importamos las librerías, denegamos alertas y seteamos la visualización del dataframe

In [0]:
import re    # regular expression nos permite editar la visualización de los caracteres según nuestras prefencias
import nltk  # natural language tokenization es una librería para manipular las palabras que encontraremos en los tweets
import string # exclusivamente para palabras 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
#Definimos un máximo de ancho de columna por la naturaleza del largo de los textos
pd.set_option("display.max_colwidth", 200)

## Entendemos la data

In [0]:
#Visualizamos el tamaño de cada dataset con el comando shape
train.shape, test.shape

La competencia nos pide predecir si un tweet es **racista o sexista** y para ello se le ha asignado un valor 1 o 0 en la columna **label** (columna objetivo en train).
Veamos cada sección de tweets según esta característica:

In [0]:
train[train['label'] == 0].head(10)

In [0]:
train[train['label'] == 1].head(10)

Podemos distinguir **cuántos** de los tweets en train son racistas o sexistas y cuántos no con un simple *value_counts* de la columna label

In [0]:
train["label"].value_counts()

Solo el 7% de los datos del dataset train contienen tweets clasificados como racista o sexista. Esto se conoce como **data desbalanceada** y existen técnicas para solucionar este problema. Pueden encontrar mayor información en [https://www.kdnuggets.com/2019/05/fix-unbalanced-dataset.html](https://)

**También podemos visualizar gráficamente el largo de los tweets** Para ello usamos *str.len* sobre cada uno de nuestros dataset: train y test

In [0]:
length_train = train['tweet'].str.len() 
length_test = test['tweet'].str.len() 
plt.hist(length_train, bins=20, label="train_tweets") #el comando hist de la librería matplotlib (plt) permite dibujar histogramas
plt.hist(length_test, bins=20, label="test_tweets") #el parámetro label es para identificar estos datos para una leyenda del gráfico
plt.legend() #el comando legend permite añadir una leyenda a nuestra gráfica
plt.show()

Podemos visualizar que el largo de los tweets de train es **significativamente mayor** que los tweets de test. El primero llega a un máximo de 6,000 caracteres por tweet, mientras que el segundo a un máximo de 3,000 caracteres por tweet

## Tratamiento de la data

En esta sección procederemos a eliminar de los datos los caracteres que nos impiden clasificar mejor los tweets (por ejemplo signos de puntuación)

In [0]:
#Inicialmente juntamos train y test para realizar este tratamiento para ambos dataset. Luego volveremos a dividirlos
combi = train.append(test, ignore_index=True) 
combi.shape

In [0]:
#Definimos una función para eliminar de los tweets alguna palabra, simbolo o caracter especial (que irá en el parámetro pattern) que creamos conveniente
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

In [0]:
#Quitamos todas aquellas menciones dentro de los tweets a otros usuarios (que comienzan con @)
combi['tidy_tweet'] = np.vectorize(remove_pattern)(combi['tweet'], "@[\w]*")  #llamamos la función remove_pattern y el parámetro pattern será @[\w]*
combi.head()

In [0]:
#Reemplazamos todos aquellos carácteres especiales como signos de puntuación por un espacio en blanco
combi['tidy_tweet'] = combi['tidy_tweet'].str.replace("[^a-zA-Z#]", " ") #el simbolo ^ signfica "todo menos" en este caso palabras y números 
combi.head(10)

In [0]:
#Removemos todas aquellas palabras con un largo de 3 caracteres o menos que podrían incluir conectores, advervios, artículos (por ejemplo a, and, the)
combi['tidy_tweet'] = combi['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
combi.head(10)

## Tokenización y Steamming

Para poder evaluar cada tweet será necesario dividirlo en palabras, este proceso se denomina *tokenization*.

También realizaremos el proceso denominado *steamming* para solo utilizar las raíces de las palabras en el análisis (por ejemplo "read" en vez de "reading")

In [0]:
#Dividimos los tweets en palabras y guardamos el resultado en una variable denominada tokenized_tweet
tokenized_tweet = combi['tidy_tweet'].apply(lambda x: x.split()) #el comando split signfica dividir
tokenized_tweet.head()

In [0]:
#Ahora procederemos a extraer solo la raíz de las palabras para un mejor análisis y actualizamos nuestra variable tokenized_tweet
from nltk.stem.porter import * 
stemmer = PorterStemmer() 
tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming

In [0]:
#Finalmente juntamos cada raíz de palabra para almacenarla nuevamente en la columna tidy_tweet
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])    
combi['tidy_tweet'] = tokenized_tweet

## Visualización de palabras más usadas y hashtag

En esta sección utilizaremos el método **wordcloud** para identificar palabras más usadas

Además visualizaremos histogramas para identificar los hashtag (#) más usados.

In [0]:
#Importamos la librería WordCloud para visualizar las palabras más usadas (que aparecen en mayor tamaño) en la columna tidy_tweet completa que denominamos all_words
from wordcloud import WordCloud
all_words = ' '.join([text for text in combi['tidy_tweet']])
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words) 
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

Repetimos el mismo análisis **solo para aquellos tweets que estén caracterizados como no racistas ni sexistas** (label=0) que denominamos normal_words

In [0]:
from wordcloud import WordCloud
normal_words =' '.join([text for text in combi['tidy_tweet'][combi['label'] == 0]]) 
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(normal_words)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

Y por último realizamos el mismo análisis para aquellos **tweets que estén caracterizados como racistas ni sexistas** (label=1) que denominamos negative_words

In [0]:
from wordcloud import WordCloud
negative_words =' '.join([text for text in combi['tidy_tweet'][combi['label'] == 1]]) 
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(negative_words)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

Para argumentar con **números** el impacto de las palabras dentro del dataset, train y test, vamos a realizar un **gráfico de barras** con ayuda del método displot.

In [0]:
# Definimos una función que nos permita extraer los hashtags (#) de los tweets
def hashtag_extract(x):
    hashtags = []
    for i in x:
        ht = re.findall(r"#(\w+)", i)
        hashtags.append(ht)
    return hashtags

In [0]:
#Extraemos los hashtags (#) de los tweets que no son racistas ni sexistas (label=0) y guardamos el resultado en la variable HT_regular
HT_regular = hashtag_extract(combi['tidy_tweet'][combi['label'] == 0])

In [0]:
#Extraemos los hashtags (#) de los tweets que sí son racistas ni sexistas (label=1) y guardamos el resultado en la variable HT_negative 
HT_negative = hashtag_extract(combi['tidy_tweet'][combi['label'] == 1])

In [0]:
#Para poder realizar un conteo y graficarlo en barras procedemos a unir todas las palabras de cada variable
HT_regular = sum(HT_regular,[])
HT_negative = sum(HT_negative,[])

In [0]:
#Con ayuda del método FreqDist de realizamos el conteo y graficamos las frecuencias de aparición de cada hashtag en los tweets de la variable HT_regular (label=0)
a = nltk.FreqDist(HT_regular)
d = pd.DataFrame({'Hashtag': list(a.keys()),
                  'Count': list(a.values())})
d = d.nlargest(columns="Count", n = 10) #Para efectos de visualización solo nos quedaremos con el top10
plt.figure(figsize=(16,5))
ax = sns.barplot(data=d, x= "Hashtag", y = "Count") #Para
ax.set(ylabel = 'Count')
plt.show()

In [0]:
#Con ayuda del método FreqDist de realizamos el conteo y graficamos las frecuencias de aparición de cada hashtag en los tweets de la variable HT_negative (label=1)
b = nltk.FreqDist(HT_negative)
e = pd.DataFrame({'Hashtag': list(b.keys()), 'Count': list(b.values())})
e = e.nlargest(columns="Count", n = 10)   #Para efectos de visualización solo nos quedaremos con el top10
plt.figure(figsize=(16,5))
ax = sns.barplot(data=e, x= "Hashtag", y = "Count")
ax.set(ylabel = 'Count')
plt.show()

## Preprocesamiento de la data ( I ): Bag of Words

El primer método para preparar la data y poder ejecutar modelos de clasificación se denomina **Bag of Words** donde se obtiene un dataset compuesto por D documentos (que representan filas o registros) y N corpus (que representan las variables predictoras)

Para nuestro ejemplo el objetivo es obtener como variables predictoras (columnas o *features*) las palabras y las filas o registros serán cada tweet.

In [0]:
#Importamos la librería CountVectorizer de sklearn
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english') #en nuestro caso estamos procesando tweets en inglés
bow = bow_vectorizer.fit_transform(combi['tidy_tweet']) #guardamos el resultado en una variable denominada bow

## Preprocesamiento de la data ( II ): TF - IDF

El segundo método para preparar la data y poder ejecutar modelos de clasificación se denomina **TF - IDF** donde se obtiene un dataset conformado por los mismos elementos. Sin embargo vamos a añadir dos elementos de evaluación (como pesos para cada variable):


1.   TF = # veces que aparece una palabra en un solo documento / # palabras totales de ese mismo documento
2.   IDF = log (N/n) n es el # veces que aparece una palabra en cada documento y N es el # total de documentos


Para nuestro ejemplo recordemos que un documento es un tweet y una palabra es cada una de las palabras que hemos tokenizado y steammeado.

In [0]:
#Importamos la librería TfidVectorizer de sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english') #en nuestro caso estamos procesando tweets en inglés
tfidf = tfidf_vectorizer.fit_transform(combi['tidy_tweet']) #guardamos el resultado en una variable denominada tfidf

## Preprocesamiento de la data ( III ): Word2Vec

El tercer método para preparar la data y poder ejecutar modelos de clasificación se denomina **Word2Vec** donde cada palabra se transforma en un vector para relacionar las palabras entre sí antes de generar el dataset. Esto nos trae dos ventajas:


1.   Se reduce significativamente la cantidad de columnas generadas
2.   Se puede brindar dos significados distintos para una misma palabra (por ejemplo *apple* puede significar fruta y también signficar empresa tecnológica)

In [0]:
#Dividimos nuevamente la columna tidy_tweet palabra por palabra para poder realizar el análisis y guardamos el resultado en la variable tokenized_tweet
tokenized_tweet = combi['tidy_tweet'].apply(lambda x: x.split())

In [0]:
#Importamos las librerías que necesitamos para ejecutar el análisis Word2vec
import gensim
from gensim.models.doc2vec import LabeledSentence

In [0]:
#Generamos un modelo word2vec y guardamos el modelo en la variable model_w2v
model_w2v = gensim.models.Word2Vec(
            tokenized_tweet,
            size=200,
            window=5,
            min_count=2,
            sg = 1,
            hs = 0,
            negative = 10,
            workers= 2,
            seed = 34)

In [0]:
#Entrenamos el modelo con los tweets tokenizados
model_w2v.train(tokenized_tweet, total_examples= len(combi['tidy_tweet']), epochs=20)

In [0]:
#Probamos el modelo entrenado para encontrar las palabras más relacionadas con la palabra "dinner"
model_w2v.wv.most_similar(positive="dinner")

In [0]:
#Definimos una función para generar un vector para cada uno de los tweet
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size))
            count += 1.
        except KeyError: continue
        if count != 0: vec/= count
    return vec

In [0]:
#Aplicamos la función sobre cada tweet y generamos un dataframe que guardaremos como la variable wordvec_df
wordvec_arrays = np.zeros((len(tokenized_tweet), 200)) 
for i in range(len(tokenized_tweet)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweet[i], 200)
    wordvec_df = pd.DataFrame(wordvec_arrays)

In [0]:
#Comprobamos el tamaño de la variable y visulizamos un total de 200 columnas
wordvec_df.shape

Este último método nos ha brindado un dataset de 200 columnas, mientras que los anteriores métodos (BagOfWords y TF-IDF) resultan en promedio 1000 columnas.

## Regresión Logística a través de Bag of Words

In [0]:
#Importamos las librerías necesarias para efectar la regresión logística
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

#Actualmente bow tiene guardado todos los tweets (ya procesados según Bag Of Words) por ello lo separamos en train y test según el shape que efectuamos al inicio
train_bow = bow[:31962,:]
test_bow = bow[31962:,:]

#Utilizamos el método train_test_split para dividir nuestra data de entrenamiento (train_bow) en train y valid
xtrain_bow, xvalid_bow, ytrain_bow, yvalid_bow = train_test_split(train_bow, train['label'], test_size=0.3)

In [0]:
#Preparamos el modelo regresión logística solo en base a los datos de entrenamiento (xtrain_bow, ytrain_bow)
lreg = LogisticRegression()
lreg.fit(xtrain_bow, ytrain_bow)

In [0]:
#Obtenemos las predicciones (probabilidades) del modelo de regresión logística utilizando los datos de validación (xvalid_bow)
prediction_bow = lreg.predict_proba(xvalid_bow)
prediction_int_bow = prediction_bow[:,1] >= 0.195 #Establecemos el punto de corte en 0.195
prediction_int_bow = prediction_int_bow.astype(np.int)

In [0]:
#Calculamos el indicador f1_score del modelo contrastando las predicciones con los target de validación (yvalid_bow)
f1_bow=f1_score(yvalid_bow, prediction_int_bow)
f1_bow

## Regresión Logística a través de TF -IDF

In [0]:
#Actualmente tfidf tiene guardado todos los tweets (ya procesados según TF-IDF) por ello lo separamos en train y test según el shape que efectuamos al inicio
train_tfidf = tfidf[:31962,:]
test_tfidf = tfidf[31962:,:]

#Utilizamos el método train_test_split para dividir nuestra data de entrenamiento (train_tfidf) en train y valid
xtrain_tfidf, xvalid_tfidf, ytrain_tfidf, yvalid_tfidf = train_test_split(train_tfidf, train['label'], test_size=0.3)

In [0]:
#Preparamos el modelo regresión logística solo en base a los datos de entrenamiento (xtrain_tfidf, ytrain_tfidf)
lreg = LogisticRegression()
lreg.fit(xtrain_tfidf, ytrain_tfidf)

In [0]:
#Obtenemos las predicciones (probabilidades) del modelo de regresión logística utilizando los datos de validación (xvalid_tfidf)
prediction_tfidf = lreg.predict_proba(xvalid_tfidf)
prediction_int_tfidf = prediction_tfidf[:,1] >= 0.295 #Establecemos el punto de corte en 0.295
prediction_int_tfidf = prediction_int_tfidf.astype(np.int)

In [0]:
#Calculamos el indicador f1_score del modelo contrastando las predicciones con los target de validación (yvalid_tfidf)
f1_tfidf=f1_score(yvalid_tfidf, prediction_int_tfidf)
f1_tfidf

## Regresión Logística a través de Word2Vec

In [0]:
#Actualmente wordvec_df tiene guardado todos los tweets (ya procesados según word2vec) por ello lo separamos en train y test según el shape que efectuamos al inicio
train_w2v = wordvec_df.iloc[:31962,:]
test_w2v = wordvec_df.iloc[31962:,:]

In [0]:
#Utilizamos el método train_test_split para dividir nuestra data de entrenamiento (train_w2v) en train y valid
xtrain_w2v, xvalid_w2v, ytrain_w2v, yvalid_w2v = train_test_split(train_w2v, train['label'], test_size=0.3)

In [0]:
#Preparamos el modelo regresión logística solo en base a los datos de entrenamiento (xtrain_w2v, ytrain_w2v)
lreg = LogisticRegression()
lreg.fit(xtrain_w2v, ytrain_w2v)

In [0]:
#Obtenemos las predicciones (probabilidades) del modelo de regresión logística utilizando los datos de validación (xvalid_w2v)
prediction_w2v = lreg.predict_proba(xvalid_w2v)
prediction_int_w2v = prediction_w2v[:,1] >= 0.1989 #Establecemos el punto de corte en 0.295
prediction_int_w2v = prediction_int_w2v.astype(np.int)

In [0]:
#Calculamos el indicador f1_score del modelo contrastando las predicciones con los target de validación (yvalid_w2v)
f1_w2v=f1_score(yvalid_w2v, prediction_int_w2v)
f1_w2v

## RandomForest

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
#Aplicamos el modelo de RandomForest para los datos procesados según Bag of Words y calculamos el indicador f1_score
rf_bow = RandomForestClassifier(n_estimators=400).fit(xtrain_bow, ytrain_bow)
prediction_rf_bow = rf_bow.predict(xvalid_bow)
f1_rf_bow=f1_score(yvalid_bow, prediction_rf_bow)
f1_rf_bow

In [0]:
#Aplicamos el modelo de RandomForest para los datos procesados según TF-IDF y calculamos el indicador f1_score
rf_tfidf = RandomForestClassifier(n_estimators=400).fit(xtrain_tfidf, ytrain_tfidf)
prediction_rf_tfidf = rf_tfidf.predict(xvalid_tfidf)
f1_rf_tfidf=f1_score(yvalid_tfidf, prediction_rf_tfidf)
f1_rf_tfidf

In [0]:
#Aplicamos el modelo de RandomForest para los datos procesados según Word2Vec y calculamos el indicador f1_score
rf_w2v = RandomForestClassifier(n_estimators=400).fit(xtrain_w2v, ytrain_w2v)
prediction_rf_w2v = rf_w2v.predict(xvalid_w2v)
f1_rf_w2v=f1_score(yvalid_w2v, prediction_rf_w2v)
f1_rf_w2v

## XGBoost

In [0]:
from xgboost import XGBClassifier

In [0]:
#Aplicamos el modelo de XGBClassifier para los datos procesados según Bag of Words y calculamos el indicador f1_score
xgbc = XGBClassifier(max_depth=6, n_estimators=1000).fit(xtrain_bow, ytrain_bow) 
prediction_xgbc_bow = xgbc.predict(xvalid_bow)
f1_xgbc_bow=f1_score(yvalid_bow, prediction_xgbc_bow)
f1_xgbc_bow

In [0]:
#Aplicamos el modelo de XGBClassifier para los datos procesados según TF-IDF y calculamos el indicador f1_score
xgbc = XGBClassifier(max_depth=6, n_estimators=1000).fit(xtrain_tfidf, ytrain_tfidf) 
prediction_xgbc_tfidf = xgbc.predict(xvalid_tfidf)
f1_xgbc_tfidf=f1_score(yvalid_tfidf, prediction_xgbc_tfidf)
f1_xgbc_tfidf

In [0]:
#Aplicamos el modelo de XGBClassifier para los datos procesados según Word2Vec y calculamos el indicador f1_score
xgbc = XGBClassifier(max_depth=6, n_estimators=1000).fit(xtrain_w2v, ytrain_w2v) 
prediction_xgbc_w2v = xgbc.predict(xvalid_w2v)
f1_xgbc_w2v=f1_score(yvalid_w2v, prediction_xgbc_w2v)
f1_xgbc_w2v

## Mejorando XGBoost + Word2Vec

In [0]:
import xgboost as xgb

In [0]:
dtrain = xgb.DMatrix(xtrain_w2v, label=ytrain_w2v) 
dvalid = xgb.DMatrix(xvalid_w2v, label=yvalid_w2v) 
dtest = xgb.DMatrix(test_w2v)
# Parameters that we are going to tune 
params = {
    'objective':'binary:logistic',
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1
 }

In [0]:
def custom_eval(preds, dtrain):
    labels = dtrain.get_label().astype(np.int)
    preds = (preds >= 0.3).astype(np.int)
    return [('f1_score', f1_score(labels, preds))]

In [0]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(6,10)
     for min_child_weight in range(5,8)
 ]

In [0]:
max_f1 = 0
best_params = None 
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight

    cv_results = xgb.cv(params,
        dtrain,        feval= custom_eval,
        num_boost_round=200,
        maximize=True,
        seed=16,
        nfold=5,
        early_stopping_rounds=10
    )
    
    mean_f1 = cv_results['test-f1_score-mean'].max()
        
    boost_rounds = cv_results['test-f1_score-mean'].argmax()    
    print("\tF1 Score {} for {} rounds".format(mean_f1, boost_rounds))
    if mean_f1 > max_f1:
      max_f1 = mean_f1
      best_params = (max_depth,min_child_weight) 
      print("Best params: {}, {}, F1 Score: {}".format(best_params[0], best_params[1], max_f1))

In [0]:
params['max_depth'] = 8 
params['min_child_weight'] = 6

In [0]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(5,10)]
    for colsample in [i/10. for i in range(5,10)] ]

In [0]:
max_f1 = 0
best_params = None 
for subsample, colsample in gridsearch_params:
    print("CV with subsample={}, colsample={}".format(subsample,colsample))
    
    params['colsample'] = colsample
    params['subsample'] = subsample
    cv_results = xgb.cv(
        params,
        dtrain,
        feval= custom_eval,
        num_boost_round=200,
        maximize=True,
        seed=16,
        nfold=5,
        early_stopping_rounds=10
    )
    
    mean_f1 = cv_results['test-f1_score-mean'].max()
    boost_rounds = cv_results['test-f1_score-mean'].argmax()
    print("\tF1 Score {} for {} rounds".format(mean_f1, boost_rounds))
    if mean_f1 > max_f1:
      max_f1 = mean_f1
      best_params = (subsample, colsample) 
      print("Best params: {}, {}, F1 Score: {}".format(best_params[0], best_params[1], max_f1))

In [0]:
params['subsample'] = .9 
params['colsample_bytree'] = .5

In [0]:
max_f1 = 0. 
best_params = None 
for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))
    
    params['eta'] = eta

    cv_results = xgb.cv(
        params,
        dtrain,
        feval= custom_eval,
        num_boost_round=1000,
        maximize=True,
        seed=16,
        nfold=5,
        early_stopping_rounds=20
    )

    mean_f1 = cv_results['test-f1_score-mean'].max()
    boost_rounds = cv_results['test-f1_score-mean'].argmax()
    print("\tF1 Score {} for {} rounds".format(mean_f1, boost_rounds))
    if mean_f1 > max_f1:
      max_f1 = mean_f1
      best_params = eta
      print("Best params: {}, F1 Score: {}".format(best_params, max_f1))

In [0]:
params['eta'] = .1

In [0]:
params
{'colsample': 0.9,
 'colsample_bytree': 0.5, 'eta': 0.1,
 'max_depth': 8, 'min_child_weight': 6,
 'objective': 'binary:logistic',
 'subsample': 0.9}

In [0]:
xgb_model = xgb.train(
    params,
    dtrain,
    feval= custom_eval,
    num_boost_round= 1000,
    maximize=True,
    evals=[(dvalid, "Validation")],
    early_stopping_rounds=10
 )

In [0]:
test_pred = xgb_model.predict(dtest)
test['label'] = (test_pred >= 0.3).astype(np.int)
submission = test[['id','label']] 
submission.to_csv('sub_xgb_w2v_finetuned.csv', index=False)